In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import tensorflow as tf
from pathlib import Path
from datetime import datetime
import argparse
from tensorflow.keras import applications
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import optimizers
from tensorflow.keras import metrics
from tensorflow.keras import Model, Input
from tensorflow.keras.applications import resnet

In [3]:
def preprocess_image(filename, target_shape=(224, 224)):
    """
    Load the specified file as a JPEG image, preprocess it and
    resize it to the target shape.
    """

    image_string = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, target_shape)
    image = resnet.preprocess_input(image)
    return image

In [11]:
# Handling data

train = np.loadtxt("handout/train_triplets.txt", dtype=str, delimiter=" ")
train = train[:2]

anchor_images = [f"handout/food/{number}.jpg" for number in list(train[:, 0])]
positive_images = [f"handout/food/{number}.jpg" for number in list(train[:, 1])]
negative_images = [f"handout/food/{number}.jpg" for number in list(train[:, 2])]

image_count = len(anchor_images)

anchor_dataset = tf.data.Dataset.from_tensor_slices(anchor_images)
positive_dataset = tf.data.Dataset.from_tensor_slices(positive_images)
negative_dataset = tf.data.Dataset.from_tensor_slices(negative_images)

anchor_dataset = anchor_dataset.map(preprocess_image)
positive_dataset = positive_dataset.map(preprocess_image)
negative_dataset = negative_dataset.map(preprocess_image)

one_anchor_dataset = anchor_dataset.take(round(.5 * image_count))
two_anchor_dataset = anchor_dataset.skip(round(.5 * image_count))
one_positive_dataset = positive_dataset.take(round(.5 * image_count))
two_positive_dataset = positive_dataset.skip(round(.5 * image_count))
one_negative_dataset = negative_dataset.take(round(.5 * image_count))
two_negative_dataset = negative_dataset.skip(round(.5 * image_count))

one_dataset = tf.data.Dataset.zip((one_anchor_dataset, one_positive_dataset, one_negative_dataset, tf.data.Dataset.from_tensor_slices(np.ones(len(one_anchor_dataset)))))
two_dataset = tf.data.Dataset.zip((two_anchor_dataset, two_negative_dataset, two_positive_dataset, tf.data.Dataset.from_tensor_slices(np.zeros(len(two_anchor_dataset)))))
dataset = one_dataset.concatenate(two_dataset)
dataset = dataset.shuffle(buffer_size=1024)

train_dataset = dataset.take(round(.8 * image_count))
validation_dataset = dataset.skip(round(.8 * image_count))

X_train = train_dataset.map(lambda anchor, positive, negative, label: (anchor, positive, negative))
X_validation = validation_dataset.map(lambda anchor, positive, negative, label: (anchor, positive, negative))
y_train = train_dataset.map(lambda anchor, positive, negative, label: label)
y_validation = validation_dataset.map(lambda anchor, positive, negative, label: label)

In [14]:
# Model

class DistanceLayer(layers.Layer):
    """
    This layer is responsible for computing the distance between the anchor
    embedding and the positive embedding, and the anchor embedding and the
    negative embedding.
    """

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, anchor, positive, negative):
        ap_distance = tf.reduce_sum(tf.square(anchor - positive), -1)
        an_distance = tf.reduce_sum(tf.square(anchor - negative), -1)
        return (ap_distance, an_distance)

target_shape = (224, 224)

base_cnn = resnet.ResNet50(
    weights="imagenet", input_shape=target_shape + (3,), include_top=False
)

flatten = layers.Flatten()(base_cnn.output)
dense1 = layers.Dense(512, activation="relu")(flatten)
dense1 = layers.BatchNormalization()(dense1)
dense2 = layers.Dense(256, activation="relu")(dense1)
dense2 = layers.BatchNormalization()(dense2)
output = layers.Dense(256)(dense2)

embedding = Model(base_cnn.input, output, name="Embedding")

trainable = False
for layer in base_cnn.layers:
    if layer.name == "conv5_block1_out":
        trainable = True
    layer.trainable = trainable

anchor_input = layers.Input(name="anchor", shape=target_shape + (3,))
positive_input = layers.Input(name="positive", shape=target_shape + (3,))
negative_input = layers.Input(name="negative", shape=target_shape + (3,))

distances = DistanceLayer()(
    embedding(anchor_input),
    embedding(positive_input),
    embedding(negative_input),
)

layer = layers.Dense(2, activation="relu")(tf.stack(distances, axis=1))
layer = layers.Dense(1, activation="sigmoid")(layer)

model = Model(inputs=[anchor_input, positive_input, negative_input], outputs=layer)

In [16]:
def train_generator():
    for anchor, positive, negative, label in train_dataset:
        yield (anchor, positive, negative), label

def validation_generator():
    for anchor, positive, negative, label in validation_dataset:
        yield (anchor, positive, negative), label

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit_generator(train_generator, epochs=10, validation_data=validation_generator)

/work/gallim/anaconda3/envs/ML-projects/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


ValueError: Failed to find data adapter that can handle input: <class 'function'>, <class 'NoneType'>

In [13]:
for i, e in zip(X_train, y_train):
    print(i, e)

2022-04-29 18:18:52.042783: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-04-29 18:18:52.043613: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2294660000 Hz


(<tf.Tensor: shape=(224, 224, 3), dtype=float32, numpy=
array([[[-103.67795 , -116.50618 , -123.29737 ],
        [-103.68668 , -116.510994, -123.31396 ],
        [-103.6582  , -116.48609 , -123.30372 ],
        ...,
        [-103.6502  , -116.50662 , -123.36517 ],
        [-103.66377 , -116.52819 , -123.391754],
        [-103.65589 , -116.5126  , -123.372955]],

       [[-103.671974, -116.500206, -123.291405],
        [-103.68593 , -116.51024 , -123.3132  ],
        [-103.64603 , -116.473915, -123.291534],
        ...,
        [-103.70965 , -116.56995 , -123.432556],
        [-103.63599 , -116.56957 , -123.42859 ],
        [-103.5733  , -116.50439 , -123.35625 ]],

       [[-103.65192 , -116.48177 , -123.26972 ],
        [-103.65596 , -116.48125 , -123.281265],
        [-103.62296 , -116.45085 , -123.26848 ],
        ...,
        [-103.6823  , -116.53818 , -123.40422 ],
        [-103.685776, -116.58491 , -123.45448 ],
        [-103.60079 , -116.49667 , -123.36617 ]],

       ...,

    